In [ ]:
import numpy as np
from numpy.polynomial.polynomial import polyfit
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

%matplotlib inline

In [ ]:
df = pd.read_csv('Todos.csv', sep=';')
df = df.drop(['Operacoes_medicoes', 'Temperatura_ensaio','Operacoes_anual', 'Nivel de atrito'], axis=1)
X = df.loc[:, ['Distancia_medicao', 'Lado', 'Remocao_anterior', 'Idade_pavimento_meses', 'Umidade_AISWEB', 'Operacoes_remocoes']]
Y = df['Atrito']

In [ ]:
df.describe().round(2)

In [ ]:
# Dividindo os conjuntos de dados entre 90% para Treinamento e 10% para Teste
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
print('Shape X_train:', X_train.shape) # Imprime a matriz dos dados de entrada de treinamento
print('Shape X_test:', X_test.shape) # Imprime a matriz dos dados de saída de treinamento
print('Shape y_train:', y_train.shape) # Imprime a matriz dos dados de entrada de teste
print('Shape y_test:', y_test.shape)  # Imprime a matriz dos dados de saída de teste

### ATENÇÃO: Na célula abaixo apenas as entradas são normalizadas

In [ ]:
# Pré-processando os dados de entrada utilizando o pré-processamento Standard Scaler
scaler_x = StandardScaler()
scaler_x.fit(X_train) # Ajustando os dados para a média e desvio padrão do conjunto de teste
X_train = scaler_x.transform(X_train) # Aplicando a transformação no conjunto de treinamento
X_test = scaler_x.transform(X_test) # O conjunto de teste é pré-processado de acordo com o conjunto de treinamento

In [ ]:
reg = MLPRegressor(
            hidden_layer_sizes=(94, 73),
            activation='relu',
            solver='lbfgs',
            alpha=0.1,
            random_state=3,
            max_iter=600)
        
reg.fit(X_train, y_train)

In [ ]:
score_treinamento = reg.score(X_train, y_train)
score_teste = reg.score(X_test, y_test)
print('R² treinamento: %.2f' %(score_treinamento*100), ' R² teste: %.2f' %(score_teste*100))

MAE_treinamento = mean_absolute_error(y_train, reg.predict(X_train))
MAE_teste = mean_absolute_error(y_test, reg.predict(X_test))
print('MAE treinamento: %.4f' %MAE_treinamento, ' MAE teste: %.4f' %MAE_teste)

MSE_treinamento = mean_squared_error(y_train, reg.predict(X_train))
MSE_teste = mean_squared_error(y_test, reg.predict(X_test))
print('MSE treinamento: %.4f' %MSE_treinamento, ' MSE teste: %.4f' %MSE_teste)

### Algumas informações acerca da Rede Neural Artificial

In [ ]:
loss = reg.loss_
coeficientes = reg.coefs_
interceptor = reg.intercepts_
n_iter = reg.n_iter_
n_iter_no_change = reg.n_iter_no_change
n_layer = reg.n_layers_
output = reg.n_outputs_
output_function = reg.out_activation_
print('Loss: %.5f\n' %loss, 
      'Iterações: %.i\n' %n_iter, 
      'Iterações sem mudança: %.i\n' %n_iter_no_change, 
      'Nº de camadas: %i\n' %n_layer,
      'Saída: %.i\n' %output,
      'Função de ativação de saída: %s' %output_function)

In [ ]:
coeficientes_entrada_1_cam_oculta = coeficientes[0]
coeficientes_1_cam_oculta_2_cam_oculta = coeficientes[1]
coeficientes_2_cam_oculta_saida = coeficientes[2]
interceptor_entrada_1_cam_oculta = interceptor[0]
interceptor_1_cam_oculta_2_cam_oculta = interceptor[1]
interceptor_2_cam_oculta_saida = interceptor[2]

### Salvandos os pesos e os bias

In [ ]:
np.savetxt('coeficientes_entrada_1_cam_oculta.txt',
           coeficientes_entrada_1_cam_oculta,
           fmt='%.4f',
           comments='# Coeficientes da camada de entrada para a 1 camada oculta')

np.savetxt('coeficientes_1_cam_oculta_2_cam_oculta.txt',
           coeficientes_1_cam_oculta_2_cam_oculta,
           fmt='%.4f',
           comments='# Coeficientes da 1 camada oculta para 2 camada oculta')

np.savetxt('coeficientes_2_cam_oculta_saida.txt',
           coeficientes_2_cam_oculta_saida,
           fmt='%.4f',
           comments='# Coeficientes da 2 camada oculta para a camada de saida')

np.savetxt('interceptor_entrada_1_cam_oculta.txt',
           interceptor_entrada_1_cam_oculta,
           fmt='%.4f',
           comments='# Interceptor da camada de entrada para a 1 camada oculta')

np.savetxt('interceptor_1_cam_oculta_2_cam_oculta.txt',
           interceptor_1_cam_oculta_2_cam_oculta,
           fmt='%.4f',
           comments='# Interceptor da 1 camada oculta para 2 camada oculta')

np.savetxt('interceptor_2_cam_oculta_saida.txt',
           interceptor_2_cam_oculta_saida,
           fmt='%.4f',
           comments='# Interceptor da 2 camada oculta para a camada de saida')

In [ ]:
plt.style.use('seaborn')# default volta ao padrão
y_train_predicted = pd.Series(reg.predict(X_train))
plt.figure(figsize=(6, 6))
plt.scatter(y_train, y_train_predicted, alpha=0.5)

plt.xlabel('Coeficiente de atrito observado', fontsize=16)
plt.ylabel('Coeficiente de atrito estimado', fontsize=16)
plt.title('Treinamento', fontsize=17)

b, m = polyfit(y_train, y_train_predicted, 1)
plt.plot(y_train, b + m * y_train, '-', color='k', alpha=0.5)
plt.text(0.5, 0.95, "y = %.3fx + %.3f"%(m, b), fontsize=14)
plt.text(0.525, 0.92, 'R2 %.2f%%' %(reg.score(X_train, y_train)*100), fontsize=14)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.savefig('Grafico_dispersao_treinamento_M2.png', dpi=300)
plt.show()

In [ ]:
y_test_predicted = pd.Series(reg.predict(X_test))
plt.figure(figsize=(6, 6))
plt.scatter(y_test, reg.predict(X_test), alpha=0.5)
plt.xlabel('Coeficiente de atrito observado', fontsize=16)
plt.ylabel('Coeficiente de atrito estimado', fontsize=16)
plt.title('Teste', fontsize=16)

b, m = polyfit(y_test, y_test_predicted, 1)
plt.plot(y_test, b + m * y_test, '-', color='k', alpha=0.5)
plt.text(0.55, 0.9, "y = %.3fx + %.3f"%(m, b), fontsize=14)
plt.text(0.575, 0.875, 'R2 %.2f%%' %(reg.score(X_test, y_test)*100), fontsize=14)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.savefig('Grafico_dispersao_teste_M2.png', dpi=300)
plt.show()

In [ ]:
erro_treinamento = pd.Series(y_train - reg.predict(X_train))
erro_teste = pd.Series(y_test - reg.predict(X_test))

In [ ]:
erro_treinamento.describe().round(2)

In [ ]:
erros_neg_treinamento = (len(erro_treinamento[erro_treinamento < 0]) / len(erro_treinamento)) * 100
erros_pos_treinamento = (len(erro_treinamento[erro_treinamento > 0]) / len(erro_treinamento)) * 100
print('Erros abaixos de 0: %.2f%%\n' %erros_neg_treinamento, 'Erros acima de 0: %.2f%%\n' %erros_pos_treinamento)

In [ ]:
plt.figure(figsize=(6, 6))
bins = np.arange(-0.25, 0.20, 0.025)
plt.hist(erro_treinamento, edgecolor='black', bins=bins) # comprimento da barra 0.03322844
plt.title('Histograma dos erros do treinamento', fontsize=18)
plt.ylabel('Frequência', fontsize=17)
plt.xlabel('Erro (observado - previsto)', fontsize=16)
plt.axvline(erro_treinamento.mean(), color='#f23843', label='Erro médio')

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=12)
plt.savefig('Histograma_erro_treinamento_M2.png', dpi=300)

In [ ]:
erro_teste.describe().round(2)

In [ ]:
erros_neg_teste = (len(erro_teste[erro_teste < 0]) / len(erro_teste)) * 100
erros_pos_teste = (len(erro_teste[erro_teste > 0]) / len(erro_teste)) * 100
print('Erros abaixos de 0: %.2f%%\n' %erros_neg_teste, 'Erros acima de 0: %.2f%%\n' %erros_pos_teste)

In [ ]:
plt.figure(figsize=(5, 5))
bins = np.arange(-0.15, 0.20, 0.025)
plt.hist(erro_teste, edgecolor='black', bins=bins) # comprimento da barra 0.02417327‬
plt.title('Histograma dos erros do teste', fontsize=16)
plt.ylabel('Frequência', fontsize=14)
plt.xlabel('Erro (observado - previsto)', fontsize=14)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.axvline(erro_teste.mean(), color='#f23843', label='Erro médio')
plt.legend()
plt.savefig('Histograma_erro_teste_M2.png', dpi=300)

### Variação das médias

In [ ]:
x_menos_1_std = pd.DataFrame([np.concatenate((4*[np.arange(200, 2600, 100)])), 
                  np.concatenate((np.zeros(24), np.ones(24), np.zeros(24), np.ones(24))), 
                  np.concatenate((np.zeros(48), np.ones(8), np.zeros(16), np.ones(8), np.zeros(16))), 
                  96*[X['Idade_pavimento_meses'].mean()-X['Idade_pavimento_meses'].std()], 
                  96*[X['Umidade_AISWEB'].mean()-X['Umidade_AISWEB'].std()], 
                  96*[X['Operacoes_remocoes'].mean()-X['Operacoes_remocoes'].std()]],
                index=X.columns)
x_menos_1_std = x_menos_1_std.T
x_menos_1_std.loc[(x_menos_1_std.Operacoes_remocoes < 0),'Operacoes_remocoes'] = 0


x_menos_2_std = pd.DataFrame([np.concatenate((4*[np.arange(200, 2600, 100)])), 
                  np.concatenate((np.zeros(24), np.ones(24), np.zeros(24), np.ones(24))), 
                  np.concatenate((np.zeros(48), np.ones(8), np.zeros(16), np.ones(8), np.zeros(16))), 
                  96*[X['Idade_pavimento_meses'].mean()-2*X['Idade_pavimento_meses'].std()], 
                  96*[X['Umidade_AISWEB'].mean()-2*X['Umidade_AISWEB'].std()], 
                  96*[X['Operacoes_remocoes'].mean()-2*X['Operacoes_remocoes'].std()]],
                index=X.columns)
x_menos_2_std = x_menos_2_std.T
x_menos_2_std.loc[(x_menos_2_std.Operacoes_remocoes < 0),'Operacoes_remocoes'] = 0


x_menos_3_std = pd.DataFrame([np.concatenate((4*[np.arange(200, 2600, 100)])), 
                  np.concatenate((np.zeros(24), np.ones(24), np.zeros(24), np.ones(24))), 
                  np.concatenate((np.zeros(48), np.ones(8), np.zeros(16), np.ones(8), np.zeros(16))), 
                  96*[X['Idade_pavimento_meses'].mean()-3*X['Idade_pavimento_meses'].std()], 
                  96*[X['Umidade_AISWEB'].mean()-3*X['Umidade_AISWEB'].std()], 
                  96*[X['Operacoes_remocoes'].mean()-3*X['Operacoes_remocoes'].std()]],
                index=X.columns)
x_menos_3_std = x_menos_3_std.T
x_menos_3_std.loc[(x_menos_3_std.Operacoes_remocoes < 0),'Operacoes_remocoes'] = 0
x_menos_3_std.loc[(x_menos_3_std.Idade_pavimento_meses < 0),'Idade_pavimento_meses'] = 0

x_mais_1_std = pd.DataFrame([np.concatenate((4*[np.arange(200, 2600, 100)])), 
                  np.concatenate((np.zeros(24), np.ones(24), np.zeros(24), np.ones(24))), 
                  np.concatenate((np.zeros(48), np.ones(8), np.zeros(16), np.ones(8), np.zeros(16))), 
                  96*[X['Idade_pavimento_meses'].mean()+X['Idade_pavimento_meses'].std()], 
                  96*[X['Umidade_AISWEB'].mean()+X['Umidade_AISWEB'].std()], 
                  96*[X['Operacoes_remocoes'].mean()+X['Operacoes_remocoes'].std()]],
                index=X.columns)
x_mais_1_std = x_mais_1_std.T
x_mais_1_std['Operacoes_remocoes'] = x_mais_1_std['Operacoes_remocoes'].map(format)

x_mais_2_std = pd.DataFrame([np.concatenate((4*[np.arange(200, 2600, 100)])), 
                  np.concatenate((np.zeros(24), np.ones(24), np.zeros(24), np.ones(24))), 
                  np.concatenate((np.zeros(48), np.ones(8), np.zeros(16), np.ones(8), np.zeros(16))), 
                  96*[X['Idade_pavimento_meses'].mean()+2*X['Idade_pavimento_meses'].std()], 
                  96*[X['Umidade_AISWEB'].mean()+2*X['Umidade_AISWEB'].std()], 
                  96*[X['Operacoes_remocoes'].mean()+2*X['Operacoes_remocoes'].std()]],
                index=X.columns)
x_mais_2_std = x_mais_2_std.T
x_mais_2_std['Operacoes_remocoes'] = x_mais_2_std['Operacoes_remocoes'].map(format)

x_mais_3_std = pd.DataFrame([np.concatenate((4*[np.arange(200, 2600, 100)])), 
                  np.concatenate((np.zeros(24), np.ones(24), np.zeros(24), np.ones(24))), 
                  np.concatenate((np.zeros(48), np.ones(8), np.zeros(16), np.ones(8), np.zeros(16))), 
                  96*[X['Idade_pavimento_meses'].mean()+3*X['Idade_pavimento_meses'].std()], 
                  96*[X['Umidade_AISWEB'].mean()+3*X['Umidade_AISWEB'].std()], 
                  96*[X['Operacoes_remocoes'].mean()+3*X['Operacoes_remocoes'].std()]],
                index=X.columns)
x_mais_3_std = x_mais_3_std.T
x_mais_3_std['Operacoes_remocoes'] = x_mais_3_std['Operacoes_remocoes'].map(format)


In [ ]:
todos_testes = pd.concat((x_menos_1_std, x_menos_2_std, x_menos_3_std, x_mais_1_std, x_mais_2_std, x_mais_3_std))

In [ ]:
# Testando com variações da média de algumas variáveis
todos_testes = scaler_x.transform(todos_testes)
y_previsto = pd.Series(reg.predict(todos_testes))

In [ ]:
y_previsto.describe().round(2)

In [ ]:
bins = np.arange(0.5, 1.05, 0.05)
plt.hist(y_previsto, ec='black', bins=bins)
plt.xlabel('Coeficiente de atrito', fontsize=16)
plt.ylabel('Frequência', fontsize=16)
plt.title('Histograma de coeficiente de atrito', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.axvline(y_previsto.mean(), label='coeficiente de atrito médio', color='red')
#plt.legend(fontsize=12)
plt.savefig('Histograma_de_coeficiente_de_atrito.png', dpi=300)

### Utilizando a Rede Neural Artificial (RNA)

**É importante que os dados de entradas estejam nessa ordem, separados por vírgula(,):**<br> 
dados_de_entrada = Distancia_medicao, Lado, Remocao_anterior, Idade_pavimento_meses, Umidade_AISWEB, Operacoes_remocoes<br>

**Explicação sobre as variáveis de entrada:**<br>
1) Distancia_medicao: As distâncias de medição devem ser preenchidas em trechos de 100m, inciando a partir de 200m até 2500m (Ex.: 200, 300, 400...);<br>

2) Lado: Os lados de medição devem ser preenchidos com 0 para o lado esquerdo e 1 para o lado direito;<br>

3) Remocao_anterior: A remoção_anterior devem ser preenchida com 0, caso não tenha sido realizada remoção do acúmulo de borracha na pista de pouso e decolagem, e 1, caso tenha ocorrido. Deve-se considerar a realização de remoção somente entre os trechos de 200m a 900m, nos demais trechos inserir 0;<br>

4) Idade_pavimento_meses: A idade do pavimento leva em consideração a quantidade de meses desde que a pista de pouso e decolagem foi recapeada até o presente momento;<br>

5) Umidade_AISWEB: Umidade relativa do ar obtida do site http://clima.icea.gov.br/clima/index.php;

6) Operacoes_remocoes: Nº de operações entre remoções, considerando pousos e decolagens, obtidos no site da ANAC;<br>

As saídas da RNA são estimativas de coeficiente de atrito medidos com GripTester a 65 km/h a 3m do eixo central da pista de pouso e decolagem.

In [ ]:
# Exemplo de aplicação
dados_entrada = pd.DataFrame([[200, 0, 0, 80, 65, 10000],[200, 1, 0, 80, 65, 10000]], columns=X.columns) # Os dados são salvos em uma variável

In [ ]:
dados_entrada # Os dados exibidos abaixo vão estimar o coeficiente de atrito no trecho de 200m, nos lados esquerdo e
              # direito, sem realização de remoção de acúmulo de borracha, considerando 80 meses de idade, umidade 
              # relativa do ar em 65% e 10.000 operações de pousos e decolagens

In [ ]:
# Faz-se o pré-processamento os dados com o Standard Scaler
dados_entrada_normalizados = scaler_x.transform(dados_entrada) # Salva-se os dados_entrada normalizados em uma nova variável

In [ ]:
coeficiente_atrito_previsto = reg.predict(dados_entrada_normalizados) # Estimando os valores de coeficiente de atrito
print('Coeficiente de atrito previsto do lado esquerdo: %.2f\n' %coeficiente_atrito_previsto[0], 
      'Coeficiente de atrito previsto do lado direito: %.2f\n' %coeficiente_atrito_previsto[1],)